In [1]:
import pandas as pd
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import re
import pathlib
from sklearn.model_selection import train_test_split
from sagemaker.tuner import IntegerParameter, HyperparameterTuner, ContinuousParameter, CategoricalParameter
from utils.utils import get_logger, load_data
from utils.load_config_file import load_config_file
from warnings import filterwarnings
filterwarnings("ignore")

logger = get_logger(name=pathlib.Path("ltv-models-training.ipynb"))

CONFIG_PATH = "config/config.ini"

pd.set_option('display.max_columns', 500)

In [16]:
## Get data from s3
s3_bucket = 'hc-data-science'
bucket_path = f"pre-conversion-ma-ltv/data/ma_ltv_merged.csv"
data_bucket_path = 's3://{}/{}'.format(s3_bucket, bucket_path)

data = pd.read_csv(data_bucket_path, low_memory=False)
data.head()

,application_id,owner_email,application_name,policy_id,owner_id,owner_phone,app_zip_code,pol_zip_code,parent_application_id,sk_referral_flag,bk_product_type,carrier,bk_gender,bk_state,lead_id,first_name,last_name,submitted_weekday,submitted_day,submitted_month,submitted_year,area_code,age,age_range,jrn_state,jrn_boberdoo_amount,jrn_boberdoo_source,jrn_boberdoo_lead_type,jrn_id,jrn_date,jrn_tracking_file_path,jrn_event_date,jrn_firstparty,jrn_tcpa_universal_id,jrn_is_first_party,jrn_error,jrn_url,jrn_year,jrn_month,jrn_day,jrn_hour,jrn_request_f_name,jrn_request_l_name,jrn_request_email,jrn_request_phone1,jrn_request_address1,jrn_request_city,jrn_request_state,jrn_request_tcpa_universal_id,jrn_request_provider,jrn_request_age,jrn_request_dob,jrn_request_smoker,jrn_request_gender,jrn_response_audit_authentic,jrn_response_audit_consumer_five_minutes,jrn_response_audit_consumer_hour,jrn_response_audit_consumer_twelve_hours,jrn_response_audit_consumer_twelve_consumer_day,jrn_response_audit_consumer_week,jrn_response_audit_data_integrity,jrn_response_audit_fields_email,jrn_response_audit_fields_f_name,jrn_response_audit_fields_l_name,jrn_response_audit_fields_phone1,jrn_response_audit_fields_city,jrn_response_audit_fields_state,jrn_response_audit_fields_address1,jrn_response_audit_device_five_minutes,jrn_response_audit_device_hour,jrn_response_audit_device_twelve_hours,jrn_response_audit_device_day,jrn_response_audit_device_week,jrn_response_audit_consumer_dupe_check,jrn_response_audit_entity_value,jrn_response_audit_ip_five_minutes,jrn_response_audit_ip_hour,jrn_response_audit_ip_twelve_hours,jrn_response_audit_ip_day,jrn_response_audit_ip_week,jrn_response_audit_lead_age,jrn_response_audit_age,jrn_response_audit_lead_duration,jrn_response_audit_duration,jrn_response_audit_lead_dupe_check,jrn_response_audit_lead_dupe,jrn_response_audit_lead_five_minutes,jrn_response_audit_lead_hour,jrn_response_audit_lead_twelve_hours,jrn_response_audit_lead_day,jrn_response_audit_lead_week,jrn_response_audit_market_leadid_tcpa_disclosure,jrn_response_audit_market_leadid_tcpa_prominence,jrn_response_audit_market_leadid_tcpa_contrast,jrn_response_audit_market_leadid_tcpa_visibility,jrn_response_audit_market_leadid_tcpa_stored,jrn_response_audit_market_leadid_tcpa_capture,jrn_response_audit_market_leadid_tcpa_result,jrn_response_audit_market_leadid_result,jrn_response_audit_market_result,jrn_response_audit_url_value,jrn_response_audit_result,jrn_response_audit_token,zcta_latitude,zcta_longitude,zcta_state,zcta_cdc_all_teeth_lost,zcta_cdc_annual_checkup,zcta_cdc_arthritis,zcta_cdc_binge_drinking,zcta_cdc_cancer_except_skin,zcta_cdc_cervical_cancer_screening,zcta_cdc_cholesterol_screening,zcta_cdc_chronic_kidney_disease,zcta_cdc_colorectal_cancer_screening,zcta_cdc_copd,zcta_cdc_core_preventive_services_for_older_men,zcta_cdc_core_preventive_services_for_older_women,zcta_cdc_coronary_heart_disease,zcta_cdc_current_asthma,zcta_cdc_current_smoking,zcta_cdc_dental_visit,zcta_cdc_depression,zcta_cdc_diabetes,zcta_cdc_general_health,zcta_cdc_health_insurance,zcta_cdc_high_blood_pressure,zcta_cdc_high_cholesterol,zcta_cdc_mammography,zcta_cdc_mental_health,zcta_cdc_obesity,zcta_cdc_physical_health,zcta_cdc_physical_inactivity,zcta_cdc_sleep_lt_7_hours,zcta_cdc_stroke,zcta_cdc_taking_bp_medication,zcta_cms_mapd_aetna_mrkt_share,zcta_cms_mapd_bcbs_mrkt_share,zcta_cms_mapd_cigna_mrkt_share,zcta_cms_mapd_humana_mrkt_share,zcta_cms_mapd_kaiser_mrkt_share,zcta_cms_mapd_lis_mrkt_share,zcta_cms_mapd_mrkt_leader,zcta_cms_mapd_other_mrkt_share,zcta_cms_mapd_penetration_2021,zcta_cms_mapd_penetration_2022,zcta_cms_mapd_penetration_pct_change,zcta_cms_mapd_priority_mrkt_share,zcta_cms_mapd_uhc_mrkt_share,zcta_cms_mapd_wellcare_mrkt_share,zcta_nyt_pct_dem,zcta_nyt_pct_gop,zcta_usc_housing_units,zcta_usc_land_area_m2,zcta_usc_median_home_value,zcta_usc_median_household_income,zcta_usc_occupied_housing_units,zcta_usc_pct_20_24,zcta_usc_pct_25_34,zcta_usc_pct_35_44,zcta_usc

In [17]:
data.pol_zip_code



0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
21295        NaN
21296        NaN
21297    77053.0
21298        NaN
21299    45154.0
Name: pol_zip_code, Length: 21300, dtype: float64

#### CatBoost Data Preprocessing

In [18]:
## Post-process the Dataset
from utils.post_processing_utils import process_catboost

X_train, X_test = process_catboost(data=data, config_path=CONFIG_PATH)


# logger.info(
#     f"Splitted input dataset shapes are: \
#         X_train= {X_train.shape}, y_train= {y_train.shape},\
#             X_test= {X_test.shape}, y_test= {y_test.shape}"
# )

In [19]:
## Also store Train and Test datasets in s3
# Train
bucket_path = f"pre-conversion-ma-ltv/data/post-processed/ma_ltv_train.csv"
train_data_path = 's3://{}/{}'.format(s3_bucket, bucket_path)
X_train.to_csv(train_data_path, index=False)
# Test
bucket_path = f"pre-conversion-ma-ltv/data/post-processed/ma_ltv_test.csv"
test_data_path = 's3://{}/{}'.format(s3_bucket, bucket_path)
X_test.to_csv(test_data_path, index=False)

In [20]:
config = load_config_file(config_path=CONFIG_PATH)
force_categorical = config["force_categorical"]
for cat in force_categorical:
    feats = [col for col in X_train.columns if cat in col]
    print(feats)

['app_zip_code']
['pol_zip_code']
['submitted_weekday', 'submitted_day', 'jrn_day', 'jrn_response_audit_consumer_twelve_consumer_day', 'jrn_response_audit_device_day', 'jrn_response_audit_ip_day', 'jrn_response_audit_lead_day']
['submitted_year', 'jrn_year']


In [24]:
print(X_train.LTV.dtype)

float64


In [21]:
experiment_name = 'rb_test1'
# s3_bucket = "s3://hc-prd-mlflow-bucket"
tracking_uri = "https://mlflow.healthcare.com/"

sess = sagemaker.Session(default_bucket=s3_bucket)
subnets = ['subnet-0b8fee7c', 'subnet-da08a7f1', 'subnet-e008a7cb']
security_group_ids = ['sg-e6d64f82']
role = 'arn:aws:iam::915124832670:role/hc-sagemaker-default-execution-role' # Local
current_user_arn = boto3.resource('iam').CurrentUser().arn # Local


In [15]:
current_user_arn

'arn:aws:iam::915124832670:user/rutvik.bhende'

In [22]:
hyperparameters = {
    'tracking_uri': tracking_uri,
    'experiment_name': experiment_name,
    'user_arn': current_user_arn,
    'target': "LTV",
    'iterations': 400,
    'learning_rate': 0.01,
    'depth': 13,
    'loss_function': "RMSE",
}

hyperparameter_ranges = {
    'iterations': IntegerParameter(200,300),
    'depth': IntegerParameter(12,13),
    'learning_rate': CategoricalParameter([0.01, 0.02]),
    'loss_function': CategoricalParameter(["RMSE", "MAE"]), 
}


metric_definitions = [
    {'Name': 'MAE_train', 'Regex': "MAE_train: ([0-9]*.[0-9]*)"},
    {'Name': 'MAE_test', 'Regex': "MAE_test: ([0-9]*.[0-9]*)"},
    {'Name': 'RMSE_train', 'Regex': "RMSE_train: ([0-9]*.[0-9]*)"},
    {'Name': 'RMSE_test', 'Regex': "RMSE_test: ([0-9]*.[0-9]*)"},
    {'Name': 'R2_score_train', 'Regex': "R2_score_train: ([0-9]*.[0-9]*)"},
    {'Name': 'R2_score_test', 'Regex': "R2_score_test: ([0-9]*.[0-9]*)"},
    {'Name': 'test_preds_mean', 'Regex': "test_preds_mean: ([0-9]*.[0-9]*)"},
]

objective_metric_name = 'RMSE_test'
objective_type = 'Minimize'

estimator = SKLearn(
    entry_point='train_catboost.py',
    source_dir='models_py',
    role=role,
    subnets=subnets,
    security_group_ids=security_group_ids,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    framework_version='0.23-1',
    py_version='py3',
)

In [23]:
estimator.fit({'train':train_data_path, 'test': test_data_path})

2022-06-10 16:54:20 Starting - Starting the training job...
2022-06-10 16:54:46 Starting - Preparing the instances for trainingProfilerReport-1654880059: InProgress
......
2022-06-10 16:55:49 Downloading - Downloading input data...
2022-06-10 16:56:10 Training - Downloading the training image...
2022-06-10 16:56:47 Training - Training image download completed. Training in progress..2022-06-10 16:56:47,779 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-06-10 16:56:47,782 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-10 16:56:47,790 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-06-10 16:56:48,635 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2022-06-10-16-54-12-922: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.errors.E

In [10]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=10,
    max_parallel_jobs=4,
    objective_type=objective_type,
    base_tuning_job_name='mlflow-ma-ltv-preconv'
)

In [11]:
tuner.fit({'train':train_data_path, 'test': test_data_path})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..................................................................................................................................................................................................................!
